In [1]:
pip show tensorflow

Name: tensorflow
Version: 2.11.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.9/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine-rl


In [2]:
import sys
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
K.clear_session()



data_entrenamiento = './drive/MyDrive/data1/entrenamiento'
data_validacion = './drive/MyDrive/data1/validacion'


In [5]:
epocas=20
longitud, altura = 150, 150
batch_size = 32
pasos = 1
validation_steps = 300
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 2
lr = 0.0004

In [6]:
##Preparamos nuestras imagenes

entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

Found 80 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [7]:
cnn = Sequential()
cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same", input_shape=(longitud, altura, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax'))

cnn.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [19]:
cnn.fit(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)


Epoch 1/20
1/1 [==============================] - 38s 38s/step - loss: 0.4451 - accuracy: 0.7812 - val_loss: 0.9492 - val_accuracy: 0.5619
Epoch 2/20
1/1 [==============================] - 33s 33s/step - loss: 0.6907 - accuracy: 0.7500 - val_loss: 1.0608 - val_accuracy: 0.5875
Epoch 4/20
1/1 [==============================] - 34s 34s/step - loss: 0.4205 - accuracy: 0.8750 - val_loss: 0.9559 - val_accuracy: 0.5999
Epoch 5/20
1/1 [==============================] - 32s 32s/step - loss: 0.5291 - accuracy: 0.8125 - val_loss: 0.8384 - val_accuracy: 0.6121
Epoch 6/20
1/1 [==============================] - 34s 34s/step - loss: 0.7279 - accuracy: 0.6875 - val_loss: 0.7730 - val_accuracy: 0.5625
Epoch 7/20
1/1 [==============================] - 33s 33s/step - loss: 0.4934 - accuracy: 0.7812 - val_loss: 0.7496 - val_accuracy: 0.5493
Epoch 8/20
1/1 [==============================] - 34s 34s/step - loss: 0.4984 - accuracy: 0.7812 - val_loss: 0.7310 - val_accuracy: 0.5879
Epoch 9/20
1/1 [===========

In [20]:
target_dir = './modelo/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')

In [10]:
def predict(file):
  
  x = load_img(file, target_size=(longitud, altura))
  
  x = img_to_array(x)
  
  x = np.expand_dims(x, axis=0)
  
  array = cnn.predict(x)
  print(array)
  result = array[0]
  print(result)
  answer = np.argmax(result)
  if answer == 0:
    print("pred: Gato")
  elif answer == 1:
    print("pred: Perro")
  
  return answer

In [21]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model


file='./drive/MyDrive/data1/cat10.jpg'
predict(file)
x = load_img(file, target_size=(longitud, altura))
x = img_to_array(x)
x = np.expand_dims(x, axis=0)
array = cnn.predict(x)




[[1. 0.]]
[1. 0.]
pred: Gato


In [12]:
target_dir = './modelo/'

longitud, altura = 150, 150
modelo = './modelo/modelo.h5'
pesos_modelo = './modelo/pesos.h5'
cnn1 = load_model(modelo)
cnn1.load_weights(pesos_modelo)

In [13]:
def predict1(file):
  
  x = load_img(file, target_size=(longitud, altura))
  
  x = img_to_array(x)
  
  x = np.expand_dims(x, axis=0)
  
  array = cnn1.predict(x)
  print(array)
  result = array[0]
  print(result)
  answer = np.argmax(result)
  if answer == 1:
    print("pred: Perro")
  elif answer == 0:
    print("pred: Gato")

  return answer

In [25]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model


file='./drive/MyDrive/data1/dog28.jpg'
predict(file)
x = load_img(file, target_size=(longitud, altura))
x = img_to_array(x)
x = np.expand_dims(x, axis=0)
array = cnn.predict(x)


[[0. 1.]]
[0. 1.]
pred: Perro
